In [ ]:
import pandas as pd
import numpy as np
#合併
# 1. 載入資料
df_est = pd.read_csv("C:\\Users\\欲合併檔名1.csv")
df_actual = pd.read_csv("C:\\Users\\欲合併檔名2.csv")

# 2. 時間欄位轉換（移除時區）
df_est['ESTSrcUpdateTime'] = pd.to_datetime(df_est['ESTSrcUpdateTime']).dt.tz_localize(None)
df_actual['ActualTime'] = pd.to_datetime(df_actual['SrcUpdateTime']).dt.tz_localize(None)

# ✅ 3. 方向欄位轉換（去程=0，回程=1，其它設為-1 並刪除）
df_actual['Direction'] = np.where(
    df_actual['Direction'] == '去程', 0,
    np.where(df_actual['Direction'] == '回程', 1, -1)
)
df_actual = df_actual[df_actual['Direction'] != -1]

# ✅ 4. 強制方向欄位型別一致（int64）
df_actual['Direction'] = df_actual['Direction'].astype('int64')
df_est['arrDirection'] = df_est['arrDirection'].astype('int64')

# 5. 排序時間欄位（merge_asof 的必要條件）
df_est = df_est.sort_values('ESTSrcUpdateTime')
df_actual = df_actual.sort_values('ActualTime')

# 6. 合併：以預估資料為主，加上實際到站時間（15分鐘容忍範圍）
df_est = pd.merge_asof(
    df_est,
    df_actual[['PlateNumb', 'StopUID', 'Direction', 'ActualTime']],
    left_by=['PlateNumb', 'arrStopUID', 'arrDirection'],
    right_by=['PlateNumb', 'StopUID', 'Direction'],
    left_on='ESTSrcUpdateTime',
    right_on='ActualTime',
    direction='forward',
    tolerance=pd.Timedelta('15m')
)

# 7. 建立新欄位：真實剩餘秒數
df_est['true_arrival_sec'] = (df_est['ActualTime'] - df_est['ESTSrcUpdateTime']).dt.total_seconds()

# 8. 移除不合理值（負數或過久）
df_est = df_est[(df_est['true_arrival_sec'] > 0) & (df_est['true_arrival_sec'] < 900)]

# 9. 儲存合併結果（選擇性）
df_est.to_csv("C:\\Users\\檔名.csv", index=False)